<a href="https://colab.research.google.com/github/CIS6930-NLP/final_project/blob/main/Finetune_T5_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Import libraries

In [1]:
!pip install --quiet datasets
!pip install --quiet transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 104.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 112.5 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
import re
import nltk
import numpy as np
import math
import time
import pandas as pd
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from datasets import load_dataset
from transformers import AutoTokenizer, TFT5ForConditionalGeneration

print(tf.__version__)

2.11.0


In [3]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


#### Loading data

In [4]:
dataset = load_dataset('aegrif/CIS6930_DAAGR_Empathetic_Dialogues')
dataset

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/10973 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/12077 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/84167 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/aegrif___parquet/aegrif--CIS6930_DAAGR_Empathetic_Dialogues-5ad2fb5e4e0762af/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    test: Dataset({
        features: ['conv_id', 'utterance_idx', 'context', 'prompt', 'utterance', 'previous_utterance', 'new_context'],
        num_rows: 10973
    })
    validation: Dataset({
        features: ['conv_id', 'utterance_idx', 'context', 'prompt', 'utterance', 'previous_utterance', 'new_context'],
        num_rows: 12077
    })
    train: Dataset({
        features: ['conv_id', 'utterance_idx', 'context', 'prompt', 'utterance', 'previous_utterance', 'new_context'],
        num_rows: 84167
    })
})

In [5]:
dataset["train"][0]

{'conv_id': 'hit:0_conv:1',
 'utterance_idx': 1,
 'context': 'sentimental',
 'prompt': 'i remember going to the fireworks with my best friend . there was a lot of people , but it only felt like us in the world .',
 'utterance': 'i remember going to see the fireworks with my best friend . it was the first time we ever spent time alone together . although there was a lot of people , we felt like the only people in the world .',
 'previous_utterance': '<|start|>',
 'new_context': 'disappointed'}

In [6]:
tr_questions = [q.strip() for q in dataset['train']["previous_utterance"]]
tr_answers = [a.strip() for a in dataset['train']["utterance"]]
print("question: " + tr_questions[1]) #list of questions
print("answer: " + tr_answers[1])

question: I remember going to see the fireworks with my best friend. It was the first time we ever spent time alone together. Although there was a lot of people, we felt like the only people in the world.
answer: was this a friend you were in love with , or just a best friend ?


#### Data preprocessing

In [7]:
##tokenize data
tokenizer = AutoTokenizer.from_pretrained("t5-small")

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [34]:
#process data
def process_data_addTask(data):
  data['input1'] = 'question: %s  context: %s </s>' % (data['previous_utterance'], data['context'])
  # data['input2'] = '%s </s>' % (data['new_context'])
  data['target'] = '%s </s>' % data['utterance']
  return data

In [35]:
# map add_eos_to_examples function to the dataset example wise 
train_dataset = dataset['train'].map(process_data_addTask)
test_dataset = dataset['test'].map(process_data_addTask)
train_dataset['target'][0]

Map:   0%|          | 0/84167 [00:00<?, ? examples/s]

Map:   0%|          | 0/10973 [00:00<?, ? examples/s]

'i remember going to see the fireworks with my best friend . it was the first time we ever spent time alone together . although there was a lot of people , we felt like the only people in the world . </s>'

In [36]:
def preprocess_function(data):
  context = data['utterance']
  question = data['input1']
  answer = data['target']
  emotion = data['context']
  

  encoder_inputs = tokenizer(question, truncation=True, 
                               return_tensors='tf', max_length=128,
                               padding="max_length")
    
  decoder_inputs = tokenizer(answer, truncation=True, 
                               return_tensors='tf', max_length=128,
                              padding="max_length")
  
  input_ids = encoder_inputs['input_ids'][0]
  # input_mask = encoder_inputs['input_ids'][0]
  input_attention = encoder_inputs['attention_mask'][0]
  target_ids = decoder_inputs['input_ids'][0]
  target_attention = decoder_inputs['attention_mask'][0]

  outputs = {'input_ids':input_ids, 'attention_mask': input_attention, 
               'labels':target_ids, 'decoder_attention_mask':target_attention}
  return outputs

In [37]:
train_ds=  train_dataset.map(preprocess_function)
test_ds=  test_dataset.map(preprocess_function)
train_ds  

Map:   0%|          | 0/84167 [00:00<?, ? examples/s]

Map:   0%|          | 0/10973 [00:00<?, ? examples/s]

Dataset({
    features: ['conv_id', 'utterance_idx', 'context', 'prompt', 'utterance', 'previous_utterance', 'new_context', 'input1', 'target', 'input_ids', 'attention_mask', 'labels', 'decoder_attention_mask'],
    num_rows: 84167
})

In [40]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator(return_tensors="tf")

In [41]:
columns = ['input_ids',  'attention_mask', 'labels', 'decoder_attention_mask']

encoded_tf_dataset = train_ds.to_tf_dataset(
    columns=columns,
    shuffle=True,
    batch_size=8,
    collate_fn=data_collator
)

encoded_tf_dataset2 = test_ds.to_tf_dataset(
    columns=columns,
   shuffle=True,
    batch_size=8,
    collate_fn=data_collator
)

In [42]:
encoded_tf_dataset

<PrefetchDataset element_spec={'labels': TensorSpec(shape=(None, 128), dtype=tf.int64, name=None), 'input_ids': TensorSpec(shape=(None, 128), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, 128), dtype=tf.int64, name=None), 'decoder_attention_mask': TensorSpec(shape=(None, 128), dtype=tf.int64, name=None)}>

In [43]:
model = TFT5ForConditionalGeneration.from_pretrained("t5-small")

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [44]:
data_dir = "/content/drive/MyDrive/code/T5_models"
log_dir = f"{data_dir}/experiments/t5/logs"
save_path = f"{data_dir}/experiments/t5/models"
cache_path_train = f"{data_dir}/cache/t5.train"
cache_path_test = f"{data_dir}/cache/t5.test"

In [45]:
import datetime
steps = 1000
start_profile_batch = steps+10
stop_profile_batch = start_profile_batch + 100
profile_range = f"{start_profile_batch},{stop_profile_batch}"

log_path = log_dir + "/" + datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_path, histogram_freq=1,
                                                     update_freq=20,profile_batch=profile_range)

checkpoint_filepath = save_path + "/" + "T5-{epoch:04d}-{val_loss:.4f}.ckpt"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

callbacks = [tensorboard_callback, model_checkpoint_callback] 
metrics = [tf.keras.metrics.SparseTopKCategoricalAccuracy(name='accuracy') ]

In [46]:
# learning_rate = CustomSchedule()
learning_rate = 0.001  # Instead set a static learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate)

In [47]:
model.compile(optimizer=optimizer, metrics=metrics)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [48]:
epochs_done = 0
model.fit(x=encoded_tf_dataset, validation_data=encoded_tf_dataset2,epochs=1)

10521/10521 [==============================] - 1949s 182ms/step - loss: 0.4692 - accuracy: 0.9469 - val_loss: 0.5036 - val_accuracy: 0.9437


### Test Model

In [49]:
context = "sentimental"
question = "what do you mean it has not been easy ? how close have you come to cheating ?"
print(context)
print(question)

sentimental
what do you mean it has not been easy ? how close have you come to cheating ?


In [50]:
input_text =  f"question: {question} context: {context} </s>"
encoded_query = tokenizer(input_text, 
                         return_tensors='tf', padding=True, truncation=True, max_length=128)
input_ids = encoded_query["input_ids"]
attention_mask = encoded_query["attention_mask"]
generated_answer = model.generate(input_ids, attention_mask=attention_mask, 
                                 max_length=128, top_p=0.95, top_k=50, repetition_penalty=float(2))
decoded_answer = tokenizer.decode(generated_answer.numpy()[0])
print("Answer: ", decoded_answer)

Answer:  <pad> i am not sure what to do with my girlfriends wife</s>


In [28]:
data_dir = "/content/drive/MyDrive/code/T5_models"
log_dir = f"{data_dir}/experiments/t5/logs"
save_path = f"{data_dir}/experiments/t5/models"
cache_path_train = f"{data_dir}/cache/t5.train"
cache_path_test = f"{data_dir}/cache/t5.test"

In [ ]:
# Save the transformer model as a SavedModel.
save_directory = "/content/drive/MyDrive/code/saved_t5_models" 
pretrained_model.save(save_directory)